In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, Model

# loading the EMOTHAW dataset\n",
import numpy as np
import pandas as pd
import glob
import os

def load_emothaw_data(directory_path):
    all_data = []
    all_labels = []

    # Use glob to find all CSV files in the specified directory
    file_paths = glob.glob(directory_path + "/*.csv")  # Modify for your file type, e.g., .npy or .txt

    for file_path in file_paths:
        # Extract the label from the filename (e.g., 'label_filename.csv')
        label = os.path.basename(file_path).split('_')[0]  # Assuming the label is the part before the first '_'

        # Load each file, assuming the first row contains the sequence length
        df = pd.read_csv(file_path, header=None)

        # Extract the sequence length from the first row
        sequence_length = int(df.iloc[0, 0])

        # Extract the feature data from the succeeding rows (excluding the first row)
        data = df.iloc[1:, :].values  # All columns are features

        # Reshape the data into (samples, sequence_length, features)
        num_samples = data.shape[0] // sequence_length
        num_features = data.shape[1]

        # Reshape the data based on the sequence length
        reshaped_data = data[:num_samples * sequence_length].reshape((num_samples, sequence_length, num_features))

        # Append the reshaped data and labels to the lists
        all_data.append(reshaped_data)

        # The same label is assigned to all samples from this file
        all_labels.append([label] * num_samples)

    # Concatenate all data and labels from the files
    all_data = np.concatenate(all_data, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    return all_data, all_labels

# Example usage: assuming your data files are stored in 'data/emothaw_files'
data, labels = load_emothaw_data('data/emothaw_files')
import numpy as np
import pandas as pd
import glob
import os

def load_emothaw_data(directory_path):
    all_data = []
    all_labels = []

    # Use glob to find all CSV files in the specified directory
    file_paths = glob.glob(directory_path + "/*.csv")  # Modify for your file type, e.g., .npy or .txt

    for file_path in file_paths:
        # Extract the label from the filename (e.g., 'label_filename.csv')
        label = os.path.basename(file_path).split('_')[0]  # Assuming the label is the part before the first '_'

        # Load each file, assuming the first row contains the sequence length
        df = pd.read_csv(file_path, header=None)

        # Extract the sequence length from the first row
        sequence_length = int(df.iloc[0, 0])

        # Extract the feature data from the succeeding rows (excluding the first row)
        data = df.iloc[1:, :].values  # All columns are features

        # Reshape the data into (samples, sequence_length, features)
        num_samples = data.shape[0] // sequence_length
        num_features = data.shape[1]

        # Reshape the data based on the sequence length
        reshaped_data = data[:num_samples * sequence_length].reshape((num_samples, sequence_length, num_features))

        # Append the reshaped data and labels to the lists
        all_data.append(reshaped_data)

        # The same label is assigned to all samples from this file
        all_labels.append([label] * num_samples)

    # Concatenate all data and labels from the files
    all_data = np.concatenate(all_data, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    return all_data, all_labels

# Example usage: assuming your data files are stored in 'data/emothaw_files'
data, labels = load_emothaw_data('data/emothaw_files')



In [ ]:
#Feature Extraction\n",
# EMOTHAW dataset, a 3D array (samples, sequence length, features)

# Extracting time domain features\n
def extract_time_domain_features(data):
    time_features = []
    for sample in data:
        sample_features = []
        for feature in sample.T:  # Assuming the features are along the last axis\n",
            feature_stats = [
                np.mean(feature),
                np.std(feature),
                np.min(feature),
                np.max(feature),
                np.percentile(feature, 25),
                np.percentile(feature, 50),
                np.percentile(feature, 75),
            ]
        time_features.append(sample_features)
    return np.array(time_features)

time_domain_features = extract_time_domain_features(data)


In [ ]:

# Extracting frequency domain features using FFT
def extract_frequency_domain_features(data):
    freq_features = []
    for sample in data:
        sample_features = []
        for feature in sample.T:  # Assuming the features are along the last axis
            freq_feature = np.fft.fft(feature)
            freq_magnitude = np.abs(freq_feature)
            dominant_freq = np.argmax(freq_magnitude)
            freq_energy = np.sum(freq_magnitude)
            sample_features.extend([dominant_freq, freq_energy])
        freq_features.append(sample_features)
    return np.array(freq_features)

frequency_domain_features = extract_frequency_domain_features(data)

In [ ]:
# Extracting statistical features
def extract_statistical_features(data):
    statistical_features = []
    for sample in data:
        sample_features = []
        for feature in sample.T:  # The features are along the last axis
            feature_stats = [
                np.mean(feature),
                np.median(feature),
                np.var(feature),
                np.skew(feature),
                np.kurtosis(feature)
            ]
            sample_features.extend(feature_stats)
        statistical_features.append(sample_features)
    return np.array(statistical_features)

statistical_features = extract_statistical_features(data)

In [ ]:
#Applying Attention-based Transformer Model
import tensorflow as tf
from tensorflow.keras import layers, Model

# Applying Attention-based Transformer Model
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}")
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:


class TransformerClassifier(Model):
    def __init__(self, num_classes, embed_dim, num_heads, ff_dim, num_layers):
        super(TransformerClassifier, self).__init__()
        self.transformer_blocks = [
            TransformerBlock(embed_dim, num_heads, ff_dim) for _ in range(num_layers)
        ]
        self.global_average_pooling = layers.GlobalAveragePooling1D()
        self.dense = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = inputs
        for transformer in self.transformer_blocks:
            x = transformer(x)
        x = self.global_average_pooling(x)
        return self.dense(x)

# Splitting the data into training and testing sets
features = np.concatenate((time_domain_features, frequency_domain_features, statistical_features), axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Defining the model parameters
num_classes = 3
embed_dim = 128
num_heads = 4
ff_dim = 128
num_layers = 2

# Creating the Transformer model
model = TransformerClassifier(num_classes, embed_dim, num_heads, ff_dim, num_layers)

# Compiling the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluating the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


In [ ]:
#Dataset splitting in Training/testing
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

# EMOTHAW dataset
def generate_sample_data(num_samples, seq_len, num_features, num_classes):
    data = np.random.rand(num_samples, seq_len, num_features)
    labels = np.random.randint(0, num_classes, num_samples)
    return data, labels

# Sample of EMOTHAW dataset\n"
num_samples = 1000
seq_length = 50
num_features = 128
num_classes = 3  # Depression, Anxiety and Stress\n",

data, labels = generate_sample_data(num_samples, seq_length, num_features, num_classes)

# Split the data into training and testing sets\n
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
"\n",
# Build and compile the model\n
num_blocks = 3
embed_dim = 64
num_heads = 4
ff_dim = 128
learning_rate = 0.001

transformer_model = TransformerClassifier(num_classes, embed_dim, num_heads, ff_dim, num_blocks)
transformer_model.compile(optimizer=Adam(learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the model
epochs = 25
batch_size = 32

transformer_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# Evaluating the model
loss, accuracy = transformer_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}")

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow import keras
import os

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3)
#model.save('Emotion-detection.model')\n",

# Compile the model\n",
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])